In [61]:

from pyspark.sql.functions import sum, when, col, lit, min, max, count, countDistinct, isnan
from pyspark.sql.functions import row_number, regexp_replace, regexp_extract, substring, concat, replace, split, explode, map_from_arrays, regexp_extract_all, coalesce
from pyspark.sql.functions import dateadd, format_number
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
import pandas as pd


In [62]:
spark = SparkSession \
    .builder \
    .appName("App Ad").getOrCreate()
#    .config("spark.some.config.option", "some-value") \

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)


In [63]:
file = "2.csv"

df = spark.read.format("csv") \
       .option("sep", ";")\
       .option("header", "false") \
       .option("inferSchema", "true") \
       .load(file) 

column_names = []
for i in range(5):
    column_names.append("set" + str(i))
# column_names = ["set1", "set2", "set3", "set4", "set5"]
df = df.toDF(*column_names)

In [64]:
print(df.count())
print(len(df.columns))


100
5


In [65]:
df.show(truncate=False)

+----------------------------------+-------------------------+-------------------------+-------------------------+--------------------------+
|set0                              |set1                     |set2                     |set3                     |set4                      |
+----------------------------------+-------------------------+-------------------------+-------------------------+--------------------------+
|Game 1: 7 red, 8 blue             | 6 blue, 6 red, 2 green  | 2 red, 6 green, 8 blue  | 9 green, 2 red, 4 blue  | 6 blue, 4 green          |
|Game 2: 2 green, 8 blue, 9 red    | 5 blue, 11 red          | 5 blue, 3 green, 4 red  | 7 blue, 8 red           | 5 red, 3 green, 3 blue   |
|Game 3: 4 red, 8 green, 2 blue    | 6 green, 2 red, 2 blue  | 12 red, 12 green, 2 blue| 10 green, 6 red         |NULL                      |
|Game 4: 12 red, 3 green           | 1 green, 2 blue, 2 red  | 16 red, 4 green         | 14 red, 3 green         | 2 blue, 5 red            |
|Game 

In [38]:
df = df.withColumn("game_no", split(df["set0"], ":")[0])
df = df.withColumn("set0", split(df["set0"], ":")[1])
df = df.withColumn('game_no',regexp_replace('game_no','Game ',"").cast(DoubleType()))

df.show(truncate=False)

+--------------------------+-------------------------+-------------------------+-------------------------+--------------------------+-------+
|set0                      |set1                     |set2                     |set3                     |set4                      |game_no|
+--------------------------+-------------------------+-------------------------+-------------------------+--------------------------+-------+
| 7 red, 8 blue            | 6 blue, 6 red, 2 green  | 2 red, 6 green, 8 blue  | 9 green, 2 red, 4 blue  | 6 blue, 4 green          |1.0    |
| 2 green, 8 blue, 9 red   | 5 blue, 11 red          | 5 blue, 3 green, 4 red  | 7 blue, 8 red           | 5 red, 3 green, 3 blue   |2.0    |
| 4 red, 8 green, 2 blue   | 6 green, 2 red, 2 blue  | 12 red, 12 green, 2 blue| 10 green, 6 red         |NULL                      |3.0    |
| 12 red, 3 green          | 1 green, 2 blue, 2 red  | 16 red, 4 green         | 14 red, 3 green         | 2 blue, 5 red            |4.0    |
| 19 r

In [39]:
# def process_game(column):
#     return (
#         explode(split(column, ";"))
#         .alias("draw")
#         .select(
#             max(when(split("draw", ",").getItem(1).contains("red"), split("draw", ",").getItem(0).cast(IntegerType())).otherwise(0)).alias("red"),
#             max(when(split("draw", ",").getItem(1).contains("green"), split("draw", ",").getItem(0).cast(IntegerType())).otherwise(0)).alias("green"),
#             max(when(split("draw", ",").getItem(1).contains("blue"), split("draw", ",").getItem(0).cast(IntegerType())).otherwise(0)).alias("blue")
#         )
#     )

# # Assuming your DataFrame 'df' has a column named 'game' containing the game data
# result = df.select(
#     process_game(col("set0")).alias("game_stats")
# )

# power_sum = result.select(
#     (col("game_stats.red") * col("game_stats.green") * col("game_stats.blue")).alias("power")
# ).agg({"power": "sum"}).collect()[0][0]

# print(power_sum)

In [43]:
# for i in column_names:
   # df = df.withColumn(str(i) + "_map", map_from_arrays(regexp_extract_all(i, lit(r"(\d+)\s(\w+)"), 2), regexp_extract_all(i, lit(r"(\d+)\s(\w+)"), 1)))

# powerSum = 0

# for row in df.collect():
#     minRed = 0
# 	minBlue = 0
# 	minGreen = 0
# 	for column in column_names:
# 		pieces = column.strip().split(',')
# 		for piece in pieces:
# 			count = int(piece.strip().split(' ')[0])
# 			color = piece.strip().split(' ')[1]
# 			if color == 'red' and count > minRed:
# 				minRed = count
# 			if color == 'blue' and count > minBlue:
# 				minBlue = count
# 			if color == 'green' and count > minGreen:
# 				minGreen = count
# 		power = minRed * minGreen * minBlue
# 		powerSum += power

# print(powerSum)

df = df.withColumn("set0_map", map_from_arrays(regexp_extract_all("set0", lit(r"(\d+)\s(\w+)"), 2), regexp_extract_all("set0", lit(r"(\d+)\s(\w+)"), 1)))
df = df.withColumn("set1_map", map_from_arrays(regexp_extract_all("set1", lit(r"(\d+)\s(\w+)"), 2), regexp_extract_all("set1", lit(r"(\d+)\s(\w+)"), 1)))
df = df.withColumn("set2_map", map_from_arrays(regexp_extract_all("set2", lit(r"(\d+)\s(\w+)"), 2), regexp_extract_all("set2", lit(r"(\d+)\s(\w+)"), 1)))
df = df.withColumn("set3_map", map_from_arrays(regexp_extract_all("set3", lit(r"(\d+)\s(\w+)"), 2), regexp_extract_all("set3", lit(r"(\d+)\s(\w+)"), 1)))
df = df.withColumn("set4_map", map_from_arrays(regexp_extract_all("set4", lit(r"(\d+)\s(\w+)"), 2), regexp_extract_all("set4", lit(r"(\d+)\s(\w+)"), 1)))

# df.show(truncate=False)

In [44]:
# Explode the map column

schema = StructType([
    StructField("game_no", DoubleType(), True), 
    StructField("blue", DoubleType(), True), 
    StructField("green", DoubleType(), True), 
    StructField("red", DoubleType(), True), 
    StructField("set", DoubleType(), True)
])
df_sets = spark.createDataFrame([], schema = schema)
schema = StructType([
    StructField("game_no", DoubleType(), True), 
    StructField("key", StringType(), True), 
    StructField("value", DoubleType(), True)
])
df_sets_exploded = spark.createDataFrame([], schema = schema)

for i in column_names:
     df_set_exploded = df.select("game_no", explode(str(i)+"_map").alias("key", "value"))
     df_set_sum = df_set_exploded.groupBy('game_no').pivot('key').agg(max("value").cast(DoubleType()).alias("total")).orderBy('game_no')
     df_set_sum = df_set_sum.withColumn('set', lit(i))
     df_sets = df_sets.union(df_set_sum)
     df_sets_exploded = df_sets_exploded.union(df_set_exploded)



# df_set1_exploded = df.select("game_no", explode("set1_map").alias("key", "value"))
# df_set2_exploded = df.select("game_no", explode("set2_map").alias("key", "value"))
# df_set3_exploded = df.select("game_no", explode("set3_map").alias("key", "value"))
# df_set4_exploded = df.select("game_no", explode("set4_map").alias("key", "value"))
# df_set5_exploded = df.select("game_no", explode("set5_map").alias("key", "value"))

# df_set1_sum = df_set1_exploded.groupBy('game_no').pivot('key').agg(sum("value").alias("total")).orderBy('game_no')
# df_set2_sum = df_set2_exploded.groupBy('game_no').pivot('key').agg(sum("value").alias("total")).orderBy('game_no')
# df_set3_sum = df_set3_exploded.groupBy('game_no').pivot('key').agg(sum("value").alias("total")).orderBy('game_no')
# df_set4_sum = df_set4_exploded.groupBy('game_no').pivot('key').agg(sum("value").alias("total")).orderBy('game_no')
# df_set5_sum = df_set5_exploded.groupBy('game_no').pivot('key').agg(sum("value").alias("total")).orderBy('game_no')

# df_set1_sum = df_set1_sum.withColumn('set', lit('1'))
# df_set2_sum = df_set2_sum.withColumn('set', lit('2'))
# df_set3_sum = df_set3_sum.withColumn('set', lit('3'))
# df_set4_sum = df_set4_sum.withColumn('set', lit('4'))
# df_set5_sum = df_set5_sum.withColumn('set', lit('5'))

# df_sets = df_set1_sum.union(df_set2_sum)
# df_sets = df_sets.union(df_set3_sum)
# df_sets = df_sets.union(df_set4_sum)
# df_sets = df_sets.union(df_set5_sum)

df_sets = df_sets.select('*').orderBy('game_no','set')
df_sets_exploded = df_sets_exploded.withColumn('value',col('value').cast(DoubleType()))
df_sets_exploded = df_sets_exploded.select('*').orderBy('game_no')

# df_sets = df_sets.fillna(0)


In [66]:
df_sets_exploded.show()

+-------+-----+-----+
|game_no|  key|value|
+-------+-----+-----+
|    1.0|  red|  7.0|
|    1.0|  red|  2.0|
|    1.0| blue|  6.0|
|    1.0|green|  6.0|
|    1.0| blue|  6.0|
|    1.0| blue|  8.0|
|    1.0|green|  4.0|
|    1.0|green|  9.0|
|    1.0|  red|  6.0|
|    1.0| blue|  8.0|
|    1.0|green|  2.0|
|    1.0|  red|  2.0|
|    1.0| blue|  4.0|
|    2.0| blue|  5.0|
|    2.0|  red|  5.0|
|    2.0|green|  3.0|
|    2.0| blue|  5.0|
|    2.0|  red|  4.0|
|    2.0|green|  3.0|
|    2.0|green|  2.0|
+-------+-----+-----+
only showing top 20 rows



In [45]:
df_sets_max = df_sets_exploded.groupBy(col('game_no'), col('key')).agg(max(col('value')).alias('value'))
df_sets_max.show(n=100)

+-------+-----+-----+
|game_no|  key|value|
+-------+-----+-----+
|   80.0|green| 10.0|
|   52.0|green| 13.0|
|   22.0|green| 10.0|
|   11.0|green| 10.0|
|   70.0|green| 12.0|
|   79.0| blue|  4.0|
|   48.0|  red|  7.0|
|   55.0| blue|  2.0|
|   76.0| blue|  9.0|
|   64.0| blue| 15.0|
|   12.0|green| 17.0|
|   32.0| blue|  3.0|
|   65.0| blue|  3.0|
|   73.0|  red|  8.0|
|   23.0|green|  6.0|
|   34.0|green| 17.0|
|   36.0| blue| 13.0|
|   54.0|green|  3.0|
|   60.0| blue|  2.0|
|    3.0|green| 12.0|
|   34.0|  red|  4.0|
|   62.0|green| 11.0|
|   25.0|  red|  3.0|
|   43.0|  red|  9.0|
|   63.0|  red| 14.0|
|   16.0|green|  9.0|
|   79.0|green|  8.0|
|   27.0|green|  5.0|
|   51.0|  red|  7.0|
|   97.0| blue|  5.0|
|    3.0| blue|  2.0|
|   11.0|  red| 14.0|
|   83.0|green|  8.0|
|   61.0|green| 18.0|
|   78.0|green|  9.0|
|   85.0|green| 11.0|
|   14.0|green|  8.0|
|    5.0|  red| 19.0|
|   16.0| blue| 12.0|
|   98.0|green|  2.0|
|    5.0|green|  2.0|
|   22.0| blue| 20.0|
|   13.0| 

In [46]:
df_sets_max_pivot = df_sets_max.groupBy(col('game_no')).pivot('key').agg(sum('value'))
df_sets_max_pivot.show(n=100)

+-------+----+-----+----+
|game_no|blue|green| red|
+-------+----+-----+----+
|   70.0|10.0| 12.0| 3.0|
|    8.0| 9.0| 11.0| 8.0|
|   67.0|15.0|  7.0|16.0|
|   69.0| 9.0|  8.0| 9.0|
|    7.0|19.0| 18.0|12.0|
|   88.0| 4.0| 13.0| 4.0|
|   49.0|11.0| 16.0|14.0|
|   98.0|12.0|  2.0| 8.0|
|   29.0|15.0| 19.0| 7.0|
|   64.0|15.0|  4.0| 1.0|
|   75.0|16.0| 20.0| 5.0|
|   47.0| 5.0|  4.0| 3.0|
|   42.0|16.0|  6.0|10.0|
|   44.0|15.0|  5.0| 1.0|
|   62.0| 2.0| 11.0| 5.0|
|   35.0| 5.0|  8.0| 5.0|
|   96.0|18.0|  1.0|12.0|
|   18.0| 8.0|  8.0|12.0|
|   80.0|14.0| 10.0|13.0|
|   86.0| 6.0|  5.0| 9.0|
|   39.0|17.0| 15.0| 6.0|
|    1.0| 8.0|  9.0| 7.0|
|   94.0| 1.0|  1.0|10.0|
|   34.0| 3.0| 17.0| 4.0|
|   37.0| 5.0| 12.0|10.0|
|   25.0| 5.0|  5.0| 3.0|
|   36.0|13.0|  9.0| 9.0|
|   41.0| 3.0|  5.0| 9.0|
|    4.0| 2.0|  4.0|16.0|
|   85.0| 2.0| 11.0| 6.0|
|   23.0|15.0|  6.0| 8.0|
|   77.0|12.0| 12.0| 9.0|
|   56.0|14.0| 14.0|14.0|
|   50.0| 2.0|  4.0| 7.0|
|   78.0|14.0|  9.0|16.0|
|   79.0| 4.

In [48]:
df_sets_max_pivot = df_sets_max_pivot.withColumn('power',col('blue') * col('green') * col('red'))
df_sets_max_pivot.show(n=100)

+-------+----+-----+----+------+
|game_no|blue|green| red| power|
+-------+----+-----+----+------+
|   70.0|10.0| 12.0| 3.0| 360.0|
|    8.0| 9.0| 11.0| 8.0| 792.0|
|   67.0|15.0|  7.0|16.0|1680.0|
|   69.0| 9.0|  8.0| 9.0| 648.0|
|    7.0|19.0| 18.0|12.0|4104.0|
|   88.0| 4.0| 13.0| 4.0| 208.0|
|   49.0|11.0| 16.0|14.0|2464.0|
|   98.0|12.0|  2.0| 8.0| 192.0|
|   29.0|15.0| 19.0| 7.0|1995.0|
|   64.0|15.0|  4.0| 1.0|  60.0|
|   75.0|16.0| 20.0| 5.0|1600.0|
|   47.0| 5.0|  4.0| 3.0|  60.0|
|   42.0|16.0|  6.0|10.0| 960.0|
|   44.0|15.0|  5.0| 1.0|  75.0|
|   62.0| 2.0| 11.0| 5.0| 110.0|
|   35.0| 5.0|  8.0| 5.0| 200.0|
|   96.0|18.0|  1.0|12.0| 216.0|
|   18.0| 8.0|  8.0|12.0| 768.0|
|   80.0|14.0| 10.0|13.0|1820.0|
|   86.0| 6.0|  5.0| 9.0| 270.0|
|   39.0|17.0| 15.0| 6.0|1530.0|
|    1.0| 8.0|  9.0| 7.0| 504.0|
|   94.0| 1.0|  1.0|10.0|  10.0|
|   34.0| 3.0| 17.0| 4.0| 204.0|
|   37.0| 5.0| 12.0|10.0| 600.0|
|   25.0| 5.0|  5.0| 3.0|  75.0|
|   36.0|13.0|  9.0| 9.0|1053.0|
|   41.0| 

In [49]:
df_sets_max_pivot.select(sum(col('power'))).show()

+----------+
|sum(power)|
+----------+
|   75534.0|
+----------+



In [36]:
# df_sets.show(n=50,truncate=False)

+-------+----+-----+----+----+
|game_no|blue|green|red |set |
+-------+----+-----+----+----+
|1.0    |8.0 |0.0  |7.0 |set0|
|1.0    |6.0 |2.0  |6.0 |set1|
|1.0    |8.0 |6.0  |2.0 |set2|
|1.0    |4.0 |9.0  |2.0 |set3|
|1.0    |6.0 |4.0  |0.0 |set4|
|2.0    |8.0 |2.0  |9.0 |set0|
|2.0    |5.0 |0.0  |11.0|set1|
|2.0    |5.0 |3.0  |4.0 |set2|
|2.0    |7.0 |0.0  |8.0 |set3|
|2.0    |3.0 |3.0  |5.0 |set4|
|3.0    |2.0 |8.0  |4.0 |set0|
|3.0    |2.0 |6.0  |2.0 |set1|
|3.0    |2.0 |12.0 |12.0|set2|
|3.0    |0.0 |10.0 |6.0 |set3|
|4.0    |0.0 |3.0  |12.0|set0|
|4.0    |2.0 |1.0  |2.0 |set1|
|4.0    |0.0 |4.0  |16.0|set2|
|4.0    |0.0 |3.0  |14.0|set3|
|4.0    |2.0 |0.0  |5.0 |set4|
|5.0    |0.0 |1.0  |19.0|set0|
|5.0    |1.0 |1.0  |7.0 |set1|
|5.0    |0.0 |0.0  |7.0 |set2|
|5.0    |0.0 |2.0  |13.0|set3|
|6.0    |2.0 |8.0  |2.0 |set0|
|6.0    |0.0 |7.0  |1.0 |set1|
|6.0    |8.0 |6.0  |3.0 |set2|
|6.0    |9.0 |14.0 |2.0 |set3|
|6.0    |0.0 |13.0 |0.0 |set4|
|7.0    |0.0 |7.0  |2.0 |set0|
|7.0    

In [52]:
df_sets = df_sets.withColumn('Fail', 
                            when(
                                (coalesce(col('blue'), lit(0)) > 14) | 
                                (coalesce(col('green'), lit(0)) > 13) | 
                                (coalesce(col('red'), lit(0)) > 12), lit(1))
                            .otherwise(lit(0))
                            )

In [38]:
# df_sets.show(n=100)

In [53]:
df_filtered = df_sets.groupBy('game_no').agg(sum('Fail')).orderBy('game_no')

In [40]:
# df_filtered.show()
# sum_when_false.show()

+-------+---------+
|game_no|sum(Fail)|
+-------+---------+
|    1.0|        0|
|    2.0|        0|
|    3.0|        0|
|    4.0|        2|
|    5.0|        2|
|    6.0|        1|
|    7.0|        3|
|    8.0|        0|
|    9.0|        1|
|   10.0|        1|
|   11.0|        1|
|   12.0|        4|
|   13.0|        1|
|   14.0|        1|
|   15.0|        1|
|   16.0|        0|
|   17.0|        0|
|   18.0|        0|
|   19.0|        1|
|   20.0|        2|
+-------+---------+
only showing top 20 rows



In [54]:
df_filtered.groupBy('sum(Fail)').agg(sum('game_no'))

sum(Fail),sum(game_no)
0,2679.0
1,1235.0
3,330.0
2,710.0
4,96.0


#### Part 2

In [55]:
# df_set1_sum = df_set1_exploded.groupBy('game_no').pivot('key').agg(sum("value").alias("total")).orderBy('game_no')
# df_set1_sum = df_set1_sum.withColumn('set', lit('1'))
# df_set2_sum = df_set2_exploded.groupBy('game_no').pivot('key').agg(sum("value").alias("total")).orderBy('game_no')
# df_set2_sum = df_set2_sum.withColumn('set', lit('2'))
# df_set3_sum = df_set3_exploded.groupBy('game_no').pivot('key').agg(sum("value").alias("total")).orderBy('game_no')
# df_set3_sum = df_set3_sum.withColumn('set', lit('3'))
# df_set4_sum = df_set4_exploded.groupBy('game_no').pivot('key').agg(sum("value").alias("total")).orderBy('game_no')
# df_set4_sum = df_set4_sum.withColumn('set', lit('4'))
# df_set5_sum = df_set5_exploded.groupBy('game_no').pivot('key').agg(sum("value").alias("total")).orderBy('game_no')
# df_set5_sum = df_set5_sum.withColumn('set', lit('5'))

# df_sets = df_set1_sum.union(df_set2_sum)
# df_sets = df_sets.union(df_set3_sum)
# df_sets = df_sets.union(df_set4_sum)
# df_sets = df_sets.union(df_set5_sum)
# df_sets = df_sets.select('*').orderBy('game_no','set')

# Sum the values
df_max = df_sets.groupBy('game_no').agg(
             max(col('blue')).alias("blue_max")
            ,max(col('green')).alias('green_max')
            ,max(col('red')).alias('red_max')
             ).orderBy('game_no')

# Show the result
df_max.show(n=100)

+-------+--------+---------+-------+
|game_no|blue_max|green_max|red_max|
+-------+--------+---------+-------+
|    1.0|     8.0|      9.0|    7.0|
|    2.0|     8.0|      3.0|   11.0|
|    3.0|     2.0|     12.0|   12.0|
|    4.0|     2.0|      4.0|   16.0|
|    5.0|     1.0|      2.0|   19.0|
|    6.0|     9.0|     14.0|    3.0|
|    7.0|    19.0|     18.0|   12.0|
|    8.0|     9.0|     11.0|    8.0|
|    9.0|    12.0|      9.0|   14.0|
|   10.0|     8.0|      1.0|   17.0|
|   11.0|    14.0|     10.0|   14.0|
|   12.0|     4.0|     17.0|    3.0|
|   13.0|    10.0|     19.0|   13.0|
|   14.0|     7.0|      8.0|   16.0|
|   15.0|    16.0|      2.0|   10.0|
|   16.0|    12.0|      9.0|   10.0|
|   17.0|     7.0|      8.0|    2.0|
|   18.0|     8.0|      8.0|   12.0|
|   19.0|    17.0|     10.0|   10.0|
|   20.0|    20.0|      9.0|   17.0|
|   21.0|     1.0|      5.0|    7.0|
|   22.0|    20.0|     10.0|   10.0|
|   23.0|    15.0|      6.0|    8.0|
|   24.0|    11.0|      7.0|    4.0|
|

In [56]:
df_max = df_max.withColumn(
    'power',
    coalesce(col('blue_max'), col('blue_max') * 1, lit(1)) * \
    coalesce(col('green_max'), col('green_max') * 1, lit(1)) * \
    coalesce(col('red_max'), col('red_max') * 1, lit(1))
)
# df_max = df_max.withColumn('power',col('blue_max') * col('green_max') * col('red_max'))
df_max.show(n=100)

+-------+--------+---------+-------+------+
|game_no|blue_max|green_max|red_max| power|
+-------+--------+---------+-------+------+
|    1.0|     8.0|      9.0|    7.0| 504.0|
|    2.0|     8.0|      3.0|   11.0| 264.0|
|    3.0|     2.0|     12.0|   12.0| 288.0|
|    4.0|     2.0|      4.0|   16.0| 128.0|
|    5.0|     1.0|      2.0|   19.0|  38.0|
|    6.0|     9.0|     14.0|    3.0| 378.0|
|    7.0|    19.0|     18.0|   12.0|4104.0|
|    8.0|     9.0|     11.0|    8.0| 792.0|
|    9.0|    12.0|      9.0|   14.0|1512.0|
|   10.0|     8.0|      1.0|   17.0| 136.0|
|   11.0|    14.0|     10.0|   14.0|1960.0|
|   12.0|     4.0|     17.0|    3.0| 204.0|
|   13.0|    10.0|     19.0|   13.0|2470.0|
|   14.0|     7.0|      8.0|   16.0| 896.0|
|   15.0|    16.0|      2.0|   10.0| 320.0|
|   16.0|    12.0|      9.0|   10.0|1080.0|
|   17.0|     7.0|      8.0|    2.0| 112.0|
|   18.0|     8.0|      8.0|   12.0| 768.0|
|   19.0|    17.0|     10.0|   10.0|1700.0|
|   20.0|    20.0|      9.0|   1

In [60]:
df_max.dtypes

[('game_no', 'double'),
 ('blue_max', 'double'),
 ('green_max', 'double'),
 ('red_max', 'double'),
 ('power', 'double')]

In [45]:
print(df_max.count())

100


In [57]:
df_max.select(sum(col('power'))).show()

+----------+
|sum(power)|
+----------+
|   75534.0|
+----------+



Mine = 75534

In [58]:
from collections import Counter
from functools import reduce
from operator import mul, or_

thres = Counter({"red":12, "green":13, "blue":14})

tot_1 = 0
tot_2 = 0
with open("input.txt", "r") as f:
    for game in f:
        game_id, draws = game.strip().split(": ")
        game_id = int(game_id.split(" ")[1])
        draws = [[c.split(" ") for c in d.split(", ")] for d in draws.split("; ")]
        draws = [Counter({c[1]:int(c[0]) for c in d}) for d in draws]
        tot_1 += all(d<=thres for d in draws) * game_id
        tot_2 += reduce(mul, reduce(or_, draws).values())
        

print(1, tot_1)
print(2, tot_2)

1 8
2 2286


In [59]:
powerSum = 0
sum = 0
with open('2.csv') as input:
	for line in input:
		game = line.strip().split(':')
		moves = game[1].split(';')
		gameID = int(game[0].strip().split(' ')[1])
		minRed = 0
		minGreen = 0
		minBlue = 0
		for move in moves:
			pieces = move.strip().split(',')
			for piece in pieces:
				count = int(piece.strip().split(' ')[0])
				color = piece.strip().split(' ')[1]
				if color == 'red' and count > minRed:
					minRed = count
				if color == 'blue' and count > minBlue:
					minBlue = count
				if color == 'green' and count > minGreen:
					minGreen = count	
		if minGreen <= 13 and minBlue <= 14 and minRed <= 12:
			sum += gameID
		power = minRed * minGreen * minBlue
		powerSum += power

print(f'Part 1 = {sum}')
print(f'Part 2 = {powerSum}')

Part 1 = 2679
Part 2 = 77607
